In [38]:
from fastai.tabular.all import *

In [39]:
df = pd.read_feather('../data/train_desc_len.feather')
df.head()

,item_condition_id,Cat1,Cat2,Cat3,Cat4,Cat5,brand_name,shipping,price,description_len
0,3,Men,Tops,T-shirts,None,None,None,1,10.0,18.0
1,3,Electronics,Computers & Tablets,Components & Parts,None,None,Razer,0,52.0,188.0
2,1,Women,Tops & Blouses,Blouse,None,None,Target,1,10.0,124.0
3,1,Home,Home Décor,Home Décor Accents,None,None,None,1,35.0,173.0
4,1,Women,Jewelry,Necklaces,None,None,None,0,44.0,41.0


In [40]:
(df.brand_name.value_counts()<1000).mean()

0.9707541257572593

In [41]:
df.brand_name.value_counts()

PINK                 53667
Nike                 53304
Victoria's Secret    47805
LuLaRoe              30848
FOREVER 21           15184
                     ...  
Dermal                   1
MeDusa                   1
Bite Beauty              1
CharmLeaks               1
Camilia                  1
Name: brand_name, Length: 4787, dtype: int64

In [42]:
unknown_brands = [name for name,count in df.brand_name.value_counts().items() if count<1000]
new_brands = {
    name: name for name in df.brand_name.unique()
}
for unknown_brand in unknown_brands:
    new_brands[unknown_brand] = 'unknown'

In [43]:
df.brand_name = df.brand_name.map(new_brands)

In [44]:
df.brand_name.value_counts()

unknown                173194
PINK                    53667
Nike                    53304
Victoria's Secret       47805
LuLaRoe                 30848
                        ...  
ColourPop Cosmetics      1030
Konami                   1026
Hello Kitty              1006
IT Cosmetics             1005
Fuji                     1003
Name: brand_name, Length: 141, dtype: int64

In [45]:
dls = TabularDataLoaders.from_df(df, y_names="price", cat_names=['item_condition_id', 'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5' , 'brand_name', 'shipping'], cont_names=['description_len'],  procs = [Categorify, FillMissing, Normalize], device=torch.device('cuda'), bs=8192)

In [48]:
learn = tabular_learner(dls, metrics=mae)
# learn.lr_find()

In [49]:
learn.fit_one_cycle(20, 1e-0)

epoch,train_loss,valid_loss,mae,time
0,243.973938,230.326202,9.803967,00:07
1,231.016693,233.627594,9.640603,00:08
2,232.301468,236.580414,10.466300,00:08
3,695.972412,305.317871,12.874602,00:07
4,262.156860,239.573807,9.976739,00:07
5,232.945206,233.089188,9.792937,00:08
6,231.903961,237.849091,10.327227,00:07
7,232.361313,231.228058,9.896018,00:08
8,229.326157,233.276993,9.536822,00:08
9,229.947235,228.853683,10.213791,00:08
